# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#отключал ограничения по выводу для дф на этапе отладки
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Список объектов

In [3]:
#задаем стартовую позицию
offset = 0

#инициализируем результирующий датафрейм
df = pd.DataFrame()

#собственно сам цикл
while True:
#     задание переменной url и параметров
    url = f'https://наш.дом.рф/сервисы/api/kn/object'
#     url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
    params = (
        ('offset',offset),
        ('limit',100),
        ('sortField','devShortCleanNm'),
        ('sortType','asc')
    )
    
    #запрос плюс преобразование результатов в джейсон и сохранение в переменную
    res = requests.get(url,params=params)
    objects_data = res.json()
    
    #получение списка Id объектов
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    
    #на каждой итерации цикла результат пишем в датафрейм
    df = df.append(pd.json_normalize(objects_data['data']['list']))
    
    #проверка на завершение списка объектов - логическое условие для прерывания цикла
    if len(objids) == 0:
        break
    
    #просто для отслеживания процесса    
#     print(len(objids),'\n')
#     print(offset)
    
    #увеличиваем позицию на максимально возможное число срок которые могут быть выведены на страницу
    offset += 100

In [4]:
#сохраняем в пикл и эксель
df.to_pickle('result.pkl')
df.to_excel('result.xlsx')

In [5]:
#создаем базу данных
engine = create_engine('sqlite://',echo=False)

In [6]:
#сохраняем датафрейм в базу данных
df.to_sql('result',con=engine)

In [ ]:
#тестовая выгрузка из базы данных
engine.execute("SELECT * FROM result").fetchall()